In [ ]:
import pandas as pd
import sqlite3

In [ ]:
!wget https://hanukkah.bluebird.sh/5783/noahs-sqlite.zip

In [ ]:
!unzip -P 5777 noahs-sqlite.zip

In [ ]:
con = sqlite3.connect("noahs.sqlite")

In [ ]:
customers = pd.read_sql(
    """
select *
from customers
""",
    con,
)

In [ ]:
orders = pd.read_sql(
    """
select *
from orders
""",
    con,
)

In [ ]:
order_items = pd.read_sql(
    """
select *
from orders_items
""",
    con,
)

In [ ]:
products = pd.read_sql(
    """
select *
from products
""",
    con,
)

In [ ]:
orders["ordered"] = pd.to_datetime(orders["ordered"])
orders["shipped"] = pd.to_datetime(orders["shipped"])
customers["customerid"] = customers["customerid"].astype(str)
order_items["orderid"] = order_items["orderid"].astype(str)

In [ ]:
def name_to_phone(name: str) -> str:
    lname = name.split(" ")[-1]
    if len(lname) != 10:
        return "BAD"
    phone = ""
    for c in lname:
        c = c.lower()
        curr = ""
        if c in ("a", "b", "c"):
            curr = "2"
        elif c in ("d", "e", "f"):
            curr = "3"
        elif c in ("g", "h", "i"):
            curr = "4"
        elif c in ("j", "k", "l"):
            curr = "5"
        elif c in ("m", "n", "o"):
            curr = "6"
        elif c in ("p", "q", "r", "s"):
            curr = "7"
        elif c in ("t", "u", "v"):
            curr = "8"
        elif c in ("w", "x", "y", "z"):
            curr = "9"
        phone += curr
    phone = phone[:3] + "-" + phone[3:6] + "-" + phone[6:]
    return phone

In [ ]:
def initials(name: str) -> str:
    names = name.split(" ")
    return names[0][0] + names[-1][0]

In [ ]:
customers["translated"] = customers["name"].apply(name_to_phone)

In [ ]:
customers["initials"] = customers["name"].apply(initials)

In [ ]:
# puzzle 1
customers[customers["translated"] == customers["phone"]]

In [ ]:
df = (
    customers.merge(orders, on="customerid")
    .merge(order_items, on="orderid")
    .merge(products, on="sku")
)

In [ ]:
# puzzle 2
orders_by_jd_in_2017 = df["initials"] == "JD"
orders_by_jd_in_2017 &= df["ordered"].dt.year == 2017
bought_these = df["desc"].str.contains("Bagel")
bought_these |= df["desc"].str.contains("Coffee")
bought_these |= df["desc"].str.contains("Clean")
(
    df[orders_by_jd_in_2017 & bought_these][
        list(customers.columns) + (list(products.columns))
    ]
    .drop_duplicates()
    .sort_values("customerid")
)

In [ ]:
def is_aries(bd: str) -> bool:
    bd_split = bd.split("-")
    y = int(bd_split[0])
    m = int(bd_split[1])
    d = int(bd_split[2])
    return (m == 3 and d >= 21) or (m == 4 and d <= 19)

In [ ]:
# birth year range?
customers["birthdate"].max(), customers["birthdate"].min()

In [ ]:
def got_that_dawg_in_em(bd: str) -> bool:
    bd_split = bd.split("-")
    y = int(bd_split[0])
    return y % 12 == 2

In [ ]:
# puzzle 3
# looking for person with the same zipcode as the contractor
aries_dog = customers["birthdate"].apply(is_aries)
aries_dog &= customers["birthdate"].apply(got_that_dawg_in_em)
neighbor = customers["citystatezip"].str.contains("11420")
customers[aries_dog & neighbor]

In [ ]:
df["shipped_date"] = df["shipped"].dt.date

In [ ]:
# puzzle 4
# do the filtering based on clues
# for each date, find the earliest pastries purchased
# could just go over the list of customers at this point
# however, can do a customer count at the end and clearly
# see one particular customer habitually buying pastries before dawn
bought_pastries = df["sku"].str.startswith("BKY")
weeks_later = "2017-04-05" <= df["shipped"].dt.date.astype(str)
before_dawn = df["shipped"].dt.time.astype(str) <= "05:00:00"
(
    df[bought_pastries & before_dawn]
    .sort_values("shipped")
    .groupby("shipped_date")
    .first()[list(customers.columns) + ["ordered", "shipped", "sku", "desc"]]
)[["name", "phone"]].value_counts().nlargest(5).reset_index()

In [ ]:
# puzzle 5
queens_village = df["citystatezip"].str.contains("Queens Village")
catfood = df["desc"].str.contains("Cat")
df[queens_village & catfood][customers.columns].drop_duplicates()

In [ ]:
df["pnl"] = (df["unit_price"] - df["wholesale_cost"]) * df["qty"]

In [ ]:
# puzzle 6
pnl_by_order = (
    df.groupby(["customerid", "name", "phone", "orderid"])
    .agg({"pnl": "sum"})
    .reset_index()
)
pnl_by_order["noah_lost_money"] = pnl_by_order["pnl"] < 0
profitability_by_customer = (
    pnl_by_order.groupby(["name", "phone"])
    .agg({"noah_lost_money": ["sum", "count"]})
    .reset_index()
)
profitability_by_customer["loss_percentage"] = (
    profitability_by_customer["noah_lost_money"]["sum"]
    / profitability_by_customer["noah_lost_money"]["count"]
)
profitability_by_customer.nlargest(10, ["loss_percentage"])

In [ ]:
# puzzle 7
bought_by_her = df["name"] == "Emily Randolph"
desc_has_color = df["desc"].str.contains("\(")
df["desc_no_color"] = df["desc"].str.split(" ").apply(lambda x: " ".join(x[:2]))

swapped_item = df[bought_by_her & desc_has_color][
    ["shipped_date", "desc_no_color", "desc"]
].merge(
    df[~bought_by_her][["shipped_date", "desc_no_color", "desc", "name", "phone"]],
    on=["shipped_date", "desc_no_color"],
)
swapped_item[swapped_item["desc_x"] != swapped_item["desc_y"]]

In [ ]:
# puzzle 8
noahs_collection = df["sku"].str.startswith("COL")
(
    df[noahs_collection]
    .drop_duplicates(["name", "phone", "sku", "desc"])
    .groupby(["name", "phone"])
    .nunique()["sku"]
    .reset_index()
    .nlargest(10, "sku")
)